In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE56814"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE56814"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE56814.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE56814.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE56814.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression study of blood monocytes in pre- and postmenopausal females with low or high bone mineral density (HuEx-1_0-st-v2)"
!Series_summary	"Comparison of circulating monocytes from pre- and postmenopausal females with low or high bone mineral density (BMD). Circulating monocytes are progenitors of osteoclasts, and produce factors important to bone metabolism. Results provide insight into the role of monocytes in osteoporosis."
!Series_summary	"We identify osteoporosis genes by microarray analyses of monocytes in high vs. low hip BMD (bone mineral density) subjects."
!Series_overall_design	"Microarray analyses of monocytes were performed using Affymetrix 1.0 ST arrays in 73 Caucasian females (age: 47-56) with extremely high (mean ZBMD =1.38, n=42, 16 pre- and 26 postmenopausal subjects) or low hip BMD (mean ZBMD=-1.05, n=31, 15 pre- and 16 postmenopausal subjects). Differential gene expression analysis in high vs. low BMD subjects was cond

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is a microarray gene expression study using Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify rows for trait, age, and gender data
trait_row = 1  # bone mineral density (BMD) is related to osteoporosis
age_row = None  # No age information is explicitly provided in the sample characteristics
gender_row = 0  # Gender information is available

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert trait (BMD) value to binary format."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 for high BMD, 0 for low BMD)
    if 'high BMD' in value.lower():
        return 1  # High BMD (not osteoporotic)
    elif 'low BMD' in value.lower():
        return 0  # Low BMD (osteoporotic)
    else:
        return None

def convert_age(value):
    """Convert age value to continuous format."""
    # This function is defined but won't be used since age data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Extract numeric value and convert to float
        age_value = ''.join(c for c in value if c.isdigit() or c == '.')
        return float(age_value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary format."""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
# Initial validation and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the clinical_data variable that should be available from previous steps
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Make sure the directory exists before saving
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1369683': [nan, 0.0], 'GSM1369684': [nan, 0.0], 'GSM1369685': [nan, 0.0], 'GSM1369686': [nan, 0.0], 'GSM1369687': [nan, 0.0], 'GSM1369688': [nan, 0.0], 'GSM1369689': [nan, 0.0], 'GSM1369690': [nan, 0.0], 'GSM1369691': [nan, 0.0], 'GSM1369692': [nan, 0.0], 'GSM1369693': [nan, 0.0], 'GSM1369694': [nan, 0.0], 'GSM1369695': [nan, 0.0], 'GSM1369696': [nan, 0.0], 'GSM1369697': [nan, 0.0], 'GSM1369698': [nan, 0.0], 'GSM1369699': [nan, 0.0], 'GSM1369700': [nan, 0.0], 'GSM1369701': [nan, 0.0], 'GSM1369702': [nan, 0.0], 'GSM1369703': [nan, 0.0], 'GSM1369704': [nan, 0.0], 'GSM1369705': [nan, 0.0], 'GSM1369706': [nan, 0.0], 'GSM1369707': [nan, 0.0], 'GSM1369708': [nan, 0.0], 'GSM1369709': [nan, 0.0], 'GSM1369710': [nan, 0.0], 'GSM1369711': [nan, 0.0], 'GSM1369712': [nan, 0.0], 'GSM1369713': [nan, 0.0], 'GSM1369714': [nan, 0.0], 'GSM1369715': [nan, 0.0], 'GSM1369716': [nan, 0.0], 'GSM1369717': [nan, 0.0], 'GSM1369718': [nan, 0.0], 'GSM1369719': [nan, 0.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (2315554, 2315633, etc.) appear to be probe IDs rather than standard human gene symbols.
# Standard gene symbols would typically be alphanumeric like BRCA1, TP53, etc.
# These numeric identifiers suggest they're likely platform-specific probe IDs that need mapping to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# The gene expression data uses numeric IDs (e.g., 2315554) which match the 'ID' column in the gene annotation
# The gene symbols are in the 'gene_assignment' column of the gene annotation

# 2. Create gene mapping dataframe using these columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene data to verify the transformation
print("Mapped gene data preview (first 10 genes):")
print(gene_data.index[:10])


Mapped gene data preview (first 10 genes):
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Debug the clinical data structure to better understand it
print("Clinical data shape:", clinical_data.shape)
print("Clinical data columns (first 5):", list(clinical_data.columns)[:5])
print("Clinical data index:", clinical_data.index.tolist())

# Let's examine one cell value completely to understand the format
sample_value = clinical_data.iloc[trait_row, 1]  # Get the second column value from the trait row
print(f"Sample trait value from clinical_data: '{sample_value}'")

# This is a transposed view of the clinical data - let's fix our approach
# The data is organized with rows as characteristics and columns as samples
# First, transpose so samples are rows
transposed_clinical_data = clinical_data.transpose()
print("Transposed clinical data shape:", transposed_clinical_data.shape)

# Get column names from the first row (the identifiers)
column_names = transposed_clinical_data.iloc[0].tolist()
# Remove the first row which was just used for headers
transposed_clinical_data = transposed_clinical_data.iloc[1:]
# Set column names
transposed_clinical_data.columns = column_names

# Now extract trait and gender data properly
trait_data = transposed_clinical_data.iloc[:, trait_row].apply(convert_trait)
gender_data = transposed_clinical_data.iloc[:, gender_row].apply(convert_gender)

# Create a proper DataFrame with the extracted features
clinical_features_df = pd.DataFrame({
    trait: trait_data,
    'Gender': gender_data
})

# Print out the fixed clinical features to verify
print("\nFixed clinical features (first 5 samples):")
print(clinical_features_df.head())
print(f"Trait values: {clinical_features_df[trait].unique()}")
print(f"Gender values: {clinical_features_df['Gender'].unique()}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = pd.merge(clinical_features_df, 
                      normalized_gene_data.transpose(), 
                      left_index=True, 
                      right_index=True)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This dataset compares monocytes from females with high versus low bone mineral density to identify genes relevant to osteoporosis."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE56814.csv
Clinical data shape: (4, 74)
Clinical data columns (first 5): ['!Sample_geo_accession', 'GSM1369683', 'GSM1369684', 'GSM1369685', 'GSM1369686']
Clinical data index: [0, 1, 2, 3]
Sample trait value from clinical_data: 'bone mineral density: high BMD'
Transposed clinical data shape: (74, 4)

Fixed clinical features (first 5 samples):
           Osteoporosis  Gender
GSM1369683         None       0
GSM1369684         None       0
GSM1369685         None       0
GSM1369686         None       0
GSM1369687         None       0
Trait values: [None]
Gender values: [0]
Clinical data saved to ../../output/preprocess/Osteoporosis/clinical_data/GSE56814.csv
Linked data shape: (73, 18420)
Quartiles for 'Osteoporosis':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Osteoporosis' in this dataset is fine.

Abnormality detected in the cohort: GSE56814. Preprocessing fai